In [18]:
import pandas as pd
import numpy as np  # Import numpy for calculations related to figure size
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import six

In [19]:
# Read the data
file_path = '../Data File Repository/Combined_Rental_Median_Price_Inventory.csv'
df = pd.read_csv(file_path)

In [20]:
# Clean and preprocess data (remove dollar signs and convert to float)
df.replace({'\$':'', ',':''}, regex=True, inplace=True)
for col in df.columns[1:]:  # Starting from 1 to skip 'Borough'
    if 'Rent' in col:  # Only columns with 'Rent' in their name are monetary values
        df[col] = df[col].astype(float)


In [21]:
# Split data into pre-COVID and post-COVID (assuming pre-COVID is before 2020)
pre_covid_years = [str(year) for year in range(2010, 2020)]
post_covid_years = [str(year) for year in range(2020, 2024)]


In [22]:
# Create a function to calculate the summary statistics
def calculate_statistics(df, years):
    summary_stats = pd.DataFrame()
    for year in years:
        rentals_col = f'{year} Total Rentals'
        median_rent_col = f'{year} Median Rent'
        summary_stats[f'{year} Avg Total Rentals'] = [df[rentals_col].mean()]
        summary_stats[f'{year} Avg Median Rent'] = [df[median_rent_col].mean()]
    summary_stats['Period'] = 'Pre-COVID' if '2010' in years else 'Post-COVID'
    return summary_stats


In [23]:
# Calculate summary statistics for pre-COVID and post-COVID periods
pre_covid_stats = calculate_statistics(df, pre_covid_years)
post_covid_stats = calculate_statistics(df, post_covid_years)


In [24]:
# Merge both summary statistics
summary_comparison = pd.concat([pre_covid_stats, post_covid_stats], ignore_index=True)


In [25]:
# Calculate changes from pre-COVID to post-COVID for each Borough
change_stats_list = []  # Use a list to store data temporarily

for borough in df['Borough'].unique():  # Make sure to use unique to avoid repetitive calculations
    borough_data = df[df['Borough'] == borough]
    pre_avg_rentals = borough_data[[year + ' Total Rentals' for year in pre_covid_years]].mean(axis=1).values[0]
    post_avg_rentals = borough_data[[year + ' Total Rentals' for year in post_covid_years]].mean(axis=1).values[0]
    pre_avg_rent = borough_data[[year + ' Median Rent' for year in pre_covid_years]].mean(axis=1).values[0]
    post_avg_rent = borough_data[[year + ' Median Rent' for year in post_covid_years]].mean(axis=1).values[0]
    
    # Append borough data to the list as a dictionary
    change_stats_list.append({
        'Borough': borough,
        'Change in Avg Rentals': post_avg_rentals - pre_avg_rentals,
        'Change in Avg Median Rent': post_avg_rent - pre_avg_rent,
        'Percentage Change in Rentals': ((post_avg_rentals - pre_avg_rentals) / pre_avg_rentals) * 100 if pre_avg_rentals else None,
        'Percentage Change in Median Rent': ((post_avg_rent - pre_avg_rent) / pre_avg_rent) * 100 if pre_avg_rent else None,
    })


In [26]:
# Convert the list of dictionaries to a DataFrame
change_stats = pd.DataFrame(change_stats_list)


In [41]:
# Format 'Change in Avg Rentals' to whole numbers
change_stats['Change in Avg Rentals'] = change_stats['Change in Avg Rentals'].astype(int)

# Format 'Change in Avg Median Rent' as currency
change_stats['Change in Avg Median Rent'] = change_stats['Change in Avg Median Rent'].apply(lambda x: "${:,.2f}".format(x))

# Format percentages to one decimal place with % symbols
change_stats['Percentage Change in Rentals'] = change_stats['Percentage Change in Rentals'].apply(lambda x: "{:.1f}%".format(x) if pd.notnull(x) else None)
change_stats['Percentage Change in Median Rent'] = change_stats['Percentage Change in Median Rent'].apply(lambda x: "{:.1f}%".format(x) if pd.notnull(x) else None)


In [42]:
# Display the title with Markdown
display(Markdown('### Change in Rentals and Median Rent by Borough (Pre vs. Post COVID-19)'))

# Display the change stats DataFrame
change_stats

### Change in Rentals and Median Rent by Borough (Pre vs. Post COVID-19)

,Borough,Change in Avg Rentals,Change in Avg Median Rent,Percentage Change in Rentals,Percentage Change in Median Rent
0,Bronx,6937,$474.92,53.4%,31.8%
1,Brooklyn,166927,$496.53,55.8%,22.8%
2,Manhattan,69812,$502.03,11.0%,15.3%
3,Queens,72425,$382.59,71.0%,21.0%
4,Staten Island,104,$608.85,17.7%,43.4%


In [43]:
# Function to render a dataframe as a table using matplotlib
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0] % len(row_colors) ])
    return ax

In [44]:
# Create a larger figure if needed
plt.figure(figsize=(25, 8))  

# Render your table to the figure
ax = render_mpl_table(change_stats, header_columns=1, col_width=4.0)  
fig = ax.get_figure()

# Set the title
fig.suptitle('Change in Rentals and Median Rent by Borough (Pre vs. Post COVID-19)', size=16)

# Adjust the layout
fig.tight_layout()

# Adjust the layout to accommodate the title
fig.subplots_adjust(top=0.9)  

# Save the figure
plt.savefig('../Charts and Graphs Output/change_stats_table.png', dpi=300, bbox_inches='tight')
plt.close(fig) 

<Figure size 2500x800 with 0 Axes>